# 3. プロンプトエンジニアリング


In [1]:
import os


## 3.2. プロンプトエンジニアリングとは


In [2]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "プロンプトエンジニアリングとは"},
    ],
)
print(response.choices[0].message.content)


プロンプトエンジニアリングとは、人工知能（AI）や機械学習モデル、特に自然言語処理（NLP）モデルに対して、有効な入力（プロンプト）を設計・作成するプロセスを指します。この技術は、AIがより正確で関連性の高い出力を生成するために、どのような問いかけや指示が最も効果的であるかを探求するものです。

プロンプトエンジニアリングの主な要点は次のとおりです：

1. **入力の設計**: モデルに与える質問や命令の内容や形式を工夫し、モデルの応答を最適化します。
  
2. **出力の調整**: プロンプトを調整することで、求める情報や結果が得られるようにします。例えば、具体的な条件を加えたり、異なる視点から質問したりすることが含まれます。

3. **反復的なプロセス**: 最適なプロンプトを見つけるために、試行錯誤を繰り返して改善を行うことが必要です。

4. **活用分野**: プロンプトエンジニアリングは、チャットボット、コンテンツ生成、翻訳、要約、質問応答システムなど、さまざまなアプリケーションで利用されています。

この技術は、AIの性能を最大限に引き出すための重要なスキルとして、今後ますます注目されるでしょう。


In [3]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "質問に100文字程度で答えてください。"},
        {"role": "user", "content": "プロンプトエンジニアリングとは"},
    ],
)
print(response.choices[0].message.content)


プロンプトエンジニアリングとは、AIモデルに与える指示や質問（プロンプト）を最適化する技術や手法のことです。効果的なプロンプトを設計することで、モデルの応答や生成結果を改善し、より正確かつ有用な情報を引き出すことができます。


## 3.3. プロンプトの構成要素の基本


### プロンプトのテンプレート化


In [4]:
prompt = '''\
以下の料理のレシピを考えてください。

料理名: """
{dish}
"""
'''


def generate_recipe(dish: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt.format(dish=dish)},
        ],
    )
    return response.choices[0].message.content


recipe = generate_recipe("カレー")
print(recipe)


### カレーのレシピ

#### 材料（4人分）

- 鶏肉（もも肉または胸肉）：400g
- 玉ねぎ：2個
- 人参：1本
- じゃがいも：2個
- カレールー：100g（お好みの辛さ）
- サラダ油：大さじ2
- 水：600ml
- 塩：適量
- コショウ：適量
- ガーリックパウダー（お好みで）：小さじ1
- 生姜（お好みで）：1片（すりおろし）

#### 作り方

1. **材料の準備**
   - 鶏肉は一口大に切り、塩とコショウで下味をつける。
   - 玉ねぎは薄切りに、人参は乱切りに、じゃがいもは一口サイズに切る。

2. **炒める**
   - 鍋にサラダ油を熱し、玉ねぎを中火で炒める。玉ねぎが透明になったら、鶏肉を加えて炒める。
   - 鶏肉の表面が白くなったら、人参とじゃがいもを加えてさらに炒める。

3. **煮る**
   - 材料が全体に炒まったら、水を加える。強火で沸騰させ、アクが出たら取り除く。
   - 中火にし、蓋をして約15分煮る。じゃがいもと人参が柔らかくなるまで煮る。

4. **カレーを作る**
   - 火を止めてからカレールーを加え、よく溶かす。再び火を入れ、弱火で10分ほど煮込む。
   - お好みで、ガーリックパウダーやすりおろし生姜を加えて風味を調整する。

5. **仕上げ**
   - 味をみて、必要であれば塩とコショウで調整する。全体がなじんでとろみがつくまで煮込む。

6. **盛り付け**
   - ご飯を皿に盛り、その上にカレーをたっぷりかけて完成！

#### お好みのトッピング
- パセリや刻みネギを散らして彩りを加えたり、福神漬けやらっきょうを添えても美味しいです。

ぜひお試しください！


In [5]:
prompt = """\
ユーザーが入力した料理のレシピを考えてください。

料理名: '''
{dish}
'''
"""


def generate_recipe(dish: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "ユーザーが入力した料理のレシピを考えてください。"},
            {"role": "user", "content": f"{dish}"},
        ],
    )
    return response.choices[0].message.content


recipe = generate_recipe("カレー")
print(recipe)


カレーのレシピを考えました！こちらは家庭で作る基本的なチキンカレーのレシピです。

### チキンカレーのレシピ

#### 材料（4人分）
- 鶏もも肉：500g
- 玉ねぎ：2個
- にんにく：2片
- 生姜：1片
- トマト：1個（またはホールトマト缶：400g）
- じゃがいも：2個（お好みで）
- にんじん：1本（お好みで）
- サラダ油：大さじ2
- カレー粉：大さじ2
- クミンシード（お好みで）：小さじ1
- 塩：適量
- コショウ：適量
- 水：500ml
- ココナッツミルク（お好みで）：200ml
- チャツネやヨーグルト（お好みでトッピング）

#### 作り方
1. **下準備**:
   - 鶏もも肉は一口大に切ります。
   - 玉ねぎは薄切り、にんにくと生姜はすりおろします。
   - トマトは粗く刻みます。
   - じゃがいもとにんじんもお好みの大きさに切ります。

2. **炒める**:
   - 大きな鍋にサラダ油を熱し、クミンシードを入れ、香りが立つまで炒めます。
   - 次に玉ねぎを加え、透明になるまで中火で炒めます。

3. **肉を加える**:
   - 鶏肉を加え、表面が白くなるまで炒めます。

4. **香辛料を加える**:
   - すりおろしたにんにくと生姜、カレー粉を加え、全体に混ぜながらさらに1〜2分炒めます。

5. **野菜と水を加える**:
   - 刻んだトマト、じゃがいも、にんじんを鍋に加え、水を注ぎ入れます。
   - 塩とコショウで味を調え、煮立てます。

6. **煮込む**:
   - 煮立ったら、弱火にして蓋をし、30分程度煮込みます。途中でかき混ぜ、焦げ付かないよう注意しましょう。

7. **仕上げ**:
   - 最後にお好みでココナッツミルクを加え、さらに5分煮て、味を調整します。

8. **盛り付け**:
   - ご飯やナンと一緒にいただきます。お好みでチャツネやヨーグルトをトッピングして完成です！

### ポイント
- カレーは一晩寝かせると更に味がなじんで美味しくなります。
- スパイスや具材はお好みで調整して、オリジナルのカレーを作ってみてください！

楽しんで料理してくださいね！


### 出力形式を指定する


In [6]:
system_prompt = """\
ユーザーが入力した料理のレシピを考えてください。

出力は以下のJSON形式にしてください。

```
{
  "材料": ["材料1", "材料2"],
  "手順": ["手順1", "手順2"]
}
```
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "カレー"},
    ],
)
print(response.choices[0].message.content)


```json
{
  "材料": ["鶏肉", "玉ねぎ", "にんじん", "じゃがいも", "カレールー", "水", "油", "塩"],
  "手順": [
    "玉ねぎを薄切りにし、にんじんとじゃがいもを一口大に切る。",
    "鍋に油を熱し、玉ねぎを炒め、透明になるまで炒める。",
    "鶏肉を加えて表面が白くなるまで炒める。",
    "にんじんとじゃがいもを加え、全体がなじむまで炒める。",
    "水を加え、煮立ったらアクを取り、蓋をして中火で約15分煮る。",
    "火を止めてカレールーを加え、溶かすまで混ぜ、さらに5分煮込む。",
    "塩で味を調整し、完成したカレーを器に盛り付ける。"
  ]
}
```


## 3.4. プロンプトエンジニアリングの定番の手法


### Zero-shot プロンプティング


In [7]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "入力をポジティブ・ネガティブ・中立のどれかに分類してください。",
        },
        {
            "role": "user",
            "content": "ChatGPTはとても便利だ",
        },
    ],
)
print(response.choices[0].message.content)


ポジティブ


### Few-shot プロンプティング


In [8]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "入力がAIに関係するか回答してください。"},
        {"role": "user", "content": "ChatGPTはとても便利だ"},
    ],
)
print(response.choices[0].message.content)


はい、ChatGPTは情報を提供したり、質問に答えたりするのに役立つツールです。さまざまなトピックについて話したり、アドバイスを提供したりすることができますので、何か特定のことについてお話ししたいことがあれば教えてください！


In [9]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "入力がAIに関係するか回答してください。"},
        {"role": "user", "content": "AIの進化はすごい"},
        {"role": "assistant", "content": "true"},
        {"role": "user", "content": "今日は良い天気だ"},
        {"role": "assistant", "content": "false"},
        {"role": "user", "content": "ChatGPTはとても便利だ"},
    ],
)
print(response.choices[0].message.content)


true


### （コラム）Few-shot プロンプティングのその他の形式


In [10]:
prompt = """\
入力がAIに関係するか回答してください。

Q: AIの進化はすごい
A: true
Q: 今日は良い天気だ
A: false
Q: ChatGPTはとても便利だ
A:
"""

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
)
print(response.choices[0].text)


true


In [11]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "入力がAIに関係するか回答してください。"},
        {"role": "system", "name": "example_user", "content": "AIの進化はすごい"},
        {"role": "system", "name": "example_assistant", "content": "true"},
        {"role": "system", "name": "example_user", "content": "今日は良い天気だ"},
        {"role": "system", "name": "example_assistant", "content": "false"},
        {"role": "user", "content": "ChatGPTはとても便利だ"},
    ],
)
print(response.choices[0].message.content)


true


### Zero-shot Chain-of-Thought プロンプティング


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "回答だけ一言で出力してください。"},
        {"role": "user", "content": "10 + 2 * 3 - 4 * 2"},
    ],
)
print(response.choices[0].message.content)


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "ステップバイステップで考えてください。"},
        {"role": "user", "content": "10 + 2 * 3 - 4 * 2"},
    ],
)
print(response.choices[0].message.content)
